In [ ]:
'''
    @Author: King
    @Date: 2019.03.18
    @Purpose: 中文文本分类实战（五）—— Bi-LSTM + Attention模型
    @Introduction:  下面介绍
    @Datasets: cnews 中文文本分类数据集。
    @Link : https://github.com/jiangxinyang227/textClassifier/tree/master/Bi-LSTM%2BAttention
    @Reference : https://www.cnblogs.com/jiangxinyang/p/10208227.html
'''

## 数据集介绍

cnews 中文文本分类数据集，其目标是判断一段话的类别，数据预处理如文本分类实战（一）—— word2vec预训练词向量中一样，预处理后的文件为../data/cnews_data/labeledCharTrain.csv。


##  Bi-LSTM + Attention 模型介绍
　  
Bi-LSTM + Attention模型来源于论文Attention-Based Bidirectional Long Short-Term Memory Networks for Relation Classification。关于Attention的介绍见这篇。

Bi-LSTM + Attention 就是在Bi-LSTM的模型上加入Attention层，<span class="burk">在Bi-LSTM中我们会用最后一个时序的输出向量 作为特征向量，然后进行softmax分类。Attention是先计算每个时序的权重，然后将所有时序 的向量进行加权和作为特征向量，然后进行softmax分类</span>。在实验中，加上Attention确实对结果有所提升。其模型结构如下图：
  
![avatar](../images/BI-LSTM+Attention.png)

In [1]:
import os
import csv
import time
import datetime
import random
import json

import warnings
from collections import Counter
from math import sqrt

import gensim
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
warnings.filterwarnings("ignore")

d:\progrom\python\python\python3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
d:\progrom\python\python\python3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# 配置参数

class TrainingConfig(object):
    epoches = 4
    evaluateEvery = 10
    checkpointEvery = 10
    learningRate = 0.001
    
class ModelConfig(object):
    embeddingSize = 200
    
    hiddenSizes = [256, 128]  # LSTM结构的神经元个数
    
    dropoutKeepProb = 0.5   
    l2RegLambda = 0.0
    
class Config(object):
    sequenceLength = 200  # 取了所有序列长度的均值
    batchSize = 128
    
    dataSource = "../data/cnews_data/labeledCharTrain.csv"
    
    stopWordSource = "../data/cnews_data/stopwords.txt"
    
    numClasses = 10
    
    rate = 0.8  # 训练集的比例
    
    training = TrainingConfig()
    
    model = ModelConfig()

    
# 实例化配置参数对象
config = Config()

## 生成训练数据

　　1）将数据加载进来，将句子分割成词表示，并去除低频词和停用词。

　　2）将词映射成索引表示，构建词汇-索引映射表，并保存成json的数据格式，之后做inference时可以用到。（注意，有的词可能不在word2vec的预训练词向量中，这种词直接用UNK表示）

　　3）从预训练的词向量模型中读取出词向量，作为初始化值输入到模型中。

　　4）将数据集分割成训练集和测试集

In [9]:
# 整块 update
import sys
# 数据预处理的类，生成训练集和测试集

class Dataset(object):
    def __init__(self, config):
        self._dataSource = config.dataSource
        self._stopWordSource = config.stopWordSource  
        
        self._sequenceLength = config.sequenceLength  # 每条输入的序列处理为定长
        self._embeddingSize = config.model.embeddingSize
        self.numClasses = config.numClasses        # update
        self._batchSize = config.batchSize
        self._rate = config.rate
        
        self._stopWordDict = {}
        
        self.trainReviews = []
        self.trainLabels = []
        
        self.evalReviews = []
        self.evalLabels = []
        
        self.wordEmbedding =None
        
        self._wordToIndex = {}
        self._indexToWord = {}
        self.label_to_index = {}
        self.index_to_label = {}
        
    def _readData(self, filePath):
        """
        从csv文件中读取数据集
        """
        
        df = pd.read_csv(filePath)
        labels = df["class"].tolist()                # update
        review = df["review_cut_word"].tolist()
        reviews = [str(line).strip().split() for line in review]

        return reviews, labels

    def _reviewProcess(self, review, sequenceLength, wordToIndex):
        """
        将数据集中的每条评论用index表示
        wordToIndex中“pad”对应的index为0
        """
        
        reviewVec = np.zeros((sequenceLength))
        sequenceLen = sequenceLength
        
        # 判断当前的序列是否小于定义的固定序列长度
        if len(review) < sequenceLength:
            sequenceLen = len(review)
            
        for i in range(sequenceLen):
            if review[i] in wordToIndex:
                reviewVec[i] = wordToIndex[review[i]]
            else:
                reviewVec[i] = wordToIndex["UNK"]

        return reviewVec

    def _labelProcess(self, y, label_to_index):
        """
        将数据集中的每个label用index表示
        label_to_index“pad”对应的index为0
        """
        input_y = []
        for item in y:
            temp = [0] * self.numClasses
            temp[label_to_index[item]] = 1
            input_y.append(temp)
        
        return input_y
    
    def _genTrainEvalData(self, x, y, rate):
        """
        生成训练集和验证集
        """
        
        reviews = []
        labels = []
        
        input_y = self._labelProcess(y,self.label_to_index)
#         print("input_y:{0}".format(input_y[0:2]))
        
#         sys.exit(0)
        
        # 遍历所有的文本，将文本中的词转换成index表示
        for i in range(len(x)):
            reviewVec = self._reviewProcess(x[i], self._sequenceLength, self._wordToIndex)
            reviews.append(reviewVec)
            
            labels.append(input_y[i])
            
        trainIndex = int(len(x) * rate)
        
        trainReviews = np.asarray(reviews[:trainIndex], dtype="int64")
        trainLabels = np.array(labels[:trainIndex], dtype="float32")
        
        evalReviews = np.asarray(reviews[trainIndex:], dtype="int64")
        evalLabels = np.array(labels[trainIndex:], dtype="float32")

        return trainReviews, trainLabels, evalReviews, evalLabels
        
    def _genVocabulary(self, reviews,labels):
        """
        生成词向量和词汇-索引映射字典，可以用全数据集
        """
        
        allWords = [word for review in reviews for word in review]
        
        # 去掉停用词
        subWords = [word for word in allWords if word not in self.stopWordDict]
        
        wordCount = Counter(subWords)  # 统计词频
        sortWordCount = sorted(wordCount.items(), key=lambda x: x[1], reverse=True)
        
        # 去除低频词
        words = [item[0] for item in sortWordCount if item[1] >= 5]
        
        vocab, wordEmbedding = self._getWordEmbedding(words)
        self.wordEmbedding = wordEmbedding
        
        self._wordToIndex = dict(zip(vocab, list(range(len(vocab)))))
        self._indexToWord = dict(zip(list(range(len(vocab))), vocab))
        

        labels = set(labels)

        for i, label in enumerate(labels):
            self.label_to_index[label] = i
            self.index_to_label[int(i)] = label
        
        
        # 将词汇-索引映射表保存为json数据，之后做inference时直接加载来处理数据
        with open("../data/wordJson/wordToIndex_cnews.json", "w", encoding="utf-8") as f:
            json.dump(self._wordToIndex, f)
        
        with open("../data/wordJson/indexToWord_cnews.json", "w", encoding="utf-8") as f:
            json.dump(self._indexToWord, f)
            
        with open('../data/wordJson/label_to_index_cnews.json', 'w', encoding='utf-8') as f:
            json.dump(self.label_to_index, f, ensure_ascii=False)

        with open('../data/wordJson/index_to_label_cnews.json', 'w', encoding='utf-8') as f:
            json.dump(self.index_to_label, f, ensure_ascii=False)
            
    def _getWordEmbedding(self, words):
        """
        按照我们的数据集中的单词取出预训练好的word2vec中的词向量
        """
        
        wordVec = gensim.models.KeyedVectors.load_word2vec_format("../data/ChnSentiCorp_cn_data/word2Vec.bin", binary=True)
        vocab = []
        wordEmbedding = []
        
        # 添加 "pad" 和 "UNK", 
        vocab.append("pad")
        vocab.append("UNK")
        wordEmbedding.append(np.zeros(self._embeddingSize))
        wordEmbedding.append(np.random.randn(self._embeddingSize))
        
        for word in words:
            try:
                vector = wordVec.wv[word]
                vocab.append(word)
                wordEmbedding.append(vector)
            except:
                #print(word + "不存在于词向量中")
                pass
                
        return vocab, np.array(wordEmbedding)
    
    def _readStopWord(self, stopWordPath):
        """
        读取停用词
        """
        
        with open(stopWordPath, "r", encoding="utf-8") as f:
            stopWords = f.read()
            stopWordList = stopWords.splitlines()
            # 将停用词用列表的形式生成，之后查找停用词时会比较快
            self.stopWordDict = dict(zip(stopWordList, list(range(len(stopWordList)))))
            
    def dataGen(self):
        """
        初始化训练集和验证集
        """
        
        # 初始化停用词
        self._readStopWord(self._stopWordSource)
        
        # 初始化数据集
        reviews, labels = self._readData(self._dataSource)
        
        # 初始化词汇-索引映射表和词向量矩阵
        self._genVocabulary(reviews,labels)
        
        # 初始化训练集和测试集
        trainReviews, trainLabels, evalReviews, evalLabels = self._genTrainEvalData(reviews, labels, self._rate)
        self.trainReviews = trainReviews
        self.trainLabels = trainLabels
        print("self.trainReviews[0:2]:{0}".format(self.trainReviews[0:3]))
        print("self.trainLabels[0:2]:{0}".format(self.trainLabels[0:3]))
        
        self.evalReviews = evalReviews
        self.evalLabels = evalLabels
        print("self.evalReviews[0:2]:{0}".format(self.evalReviews[0:3]))
        print("self.evalLabels[0:2]:{0}".format(self.evalLabels[0:3]))
        
        
data = Dataset(config)
data.dataGen()

self.trainReviews[0:2]:[[   1    1    1    1    1    1    1    1    1    1    1    1 1131    1
     1    1    1  478  142    1   27    2    1  483    1    1    1    1
     1    1 1281    1    1    1    1    1    1  886    1    1    1    1
     1    1    1    1    1    1 1005    1    1    1   13    1    1    1
     1    1    1    1    1    1    1  203  215    1    1    1    1    1
     1    1   27    1    1    1    1    1  102    1  152    1    1    1
     1  451    1    1  753    1    1  226    1    1   65    1    1    1
     1    1    1   19    1    1    1    1    1  849    1    1 2243    1
     1    1    1    1    1    1    1    1  878    1    1    1    1    1
     1    1    1    1    1    1    1    1    1    1    1  419    1    1
   196    1    1    1    1    1    1    1    1    1    1    1    1    1
     1    1   10    1    1    8    1    1 1654    1    1    1    1    1
     1    1    1   36    1    1    1 1151    1    1    1    1    1    1
     1    1  120    1    1 1162    1    1

In [10]:
print("train data shape: {}".format(data.trainReviews.shape))
print("train label shape: {}".format(data.trainLabels.shape))
print("eval data shape: {}".format(data.evalReviews.shape))
print("wordEmbedding data shape: {}".format(data.wordEmbedding.shape))
print("_wordToIndex data shape: {}".format(len(data._wordToIndex)))
print("_indexToWord data shape: {}".format(len(data._indexToWord)))

train data shape: (3999, 200)
train label shape: (3999, 10)
eval data shape: (1000, 200)
wordEmbedding data shape: (3302, 200)
_wordToIndex data shape: 3302
_indexToWord data shape: 3302


## 生成batch数据集

　　采用生成器的形式向模型输入batch数据集，（生成器可以避免将所有的数据加入到内存中）

In [11]:
# 输出batch数据集

def nextBatch(x, y, batchSize):
        '''
         生成batch数据集，用生成器的方式输出
        
        :param x:      list    
        :param y:      list    
        :param batchSize:      int    
        :return: 
            batchX        nnumpy array
            batchY        nnumpy array
            
        '''
    
        perm = np.arange(len(x))
        np.random.shuffle(perm)
        x = x[perm]
        y = y[perm]
        
        numBatches = len(x) // batchSize

        for i in range(numBatches):
            start = i * batchSize
            end = start + batchSize
            batchX = np.array(x[start: end], dtype="int64")
            batchY = np.array(y[start: end], dtype="float32")
            
            yield batchX, batchY

## Bi-LSTM + Attention模型

In [12]:
# 构建模型
class BiLSTMAttention(object):
    """
    Bi-LSTM + Attention模型 用于文本分类
    """
    def __init__(self, config, wordEmbedding):

        # 定义模型的输入
        self.numClasses = config.numClasses      # update
        self.inputX = tf.placeholder(tf.int32, [None, config.sequenceLength], name="inputX")
        self.inputY = tf.placeholder(tf.float32, [None, self.numClasses], name="inputY")
        
        self.dropoutKeepProb = tf.placeholder(tf.float32, name="dropoutKeepProb")
        
        # 定义l2损失
        l2Loss = tf.constant(0.0)
        
        # 词嵌入层
        with tf.name_scope("embedding"):

            # 利用预训练的词向量初始化词嵌入矩阵
            self.W = tf.Variable(tf.cast(wordEmbedding, dtype=tf.float32, name="word2vec") ,name="W")
            # 利用词嵌入矩阵将输入的数据中的词转换成词向量，维度[batch_size, sequence_length, embedding_size]
            self.embeddedWords = tf.nn.embedding_lookup(self.W, self.inputX)
            
        # 定义两层双向LSTM的模型结构
        with tf.name_scope("Bi-LSTM"):
            for idx, hiddenSize in enumerate(config.model.hiddenSizes):
                with tf.name_scope("Bi-LSTM" + str(idx)):
                    # 定义前向LSTM结构
                    lstmFwCell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
                                                                 output_keep_prob=self.dropoutKeepProb)
                    # 定义反向LSTM结构
                    lstmBwCell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
                                                                 output_keep_prob=self.dropoutKeepProb)


                    # 采用动态rnn，可以动态的输入序列的长度，若没有输入，则取序列的全长
                    # outputs是一个元祖(output_fw, output_bw)，其中两个元素的维度都是[batch_size, max_time, hidden_size],fw和bw的hidden_size一样
                    # self.current_state 是最终的状态，二元组(state_fw, state_bw)，state_fw=[batch_size, s]，s是一个元祖(h, c)
                    outputs_, self.current_state = tf.nn.bidirectional_dynamic_rnn(lstmFwCell, lstmBwCell, 
                                                                                  self.embeddedWords, dtype=tf.float32,
                                                                                  scope="bi-lstm" + str(idx))
        
                    # 对outputs中的fw和bw的结果拼接 [batch_size, time_step, hidden_size * 2], 传入到下一层Bi-LSTM中
                    self.embeddedWords = tf.concat(outputs_, 2)
                
        # 将最后一层Bi-LSTM输出的结果分割成前向和后向的输出
        outputs = tf.split(self.embeddedWords, 2, -1)
        
        # 在Bi-LSTM+Attention的论文中，将前向和后向的输出相加
        with tf.name_scope("Attention"):
            H = outputs[0] + outputs[1]

            # 得到Attention的输出
            output = self.attention(H)
            outputSize = config.model.hiddenSizes[-1]
        
        # 全连接层的输出
        with tf.name_scope("output"):
            outputW = tf.get_variable(
                "outputW",
                shape=[outputSize, self.numClasses],
                initializer=tf.contrib.layers.xavier_initializer())
            
            outputB= tf.Variable(tf.constant(0.1, shape=[self.numClasses]), name="outputB")
            l2Loss += tf.nn.l2_loss(outputW)
            l2Loss += tf.nn.l2_loss(outputB)
            
            # update
#             self.predictions = tf.nn.xw_plus_b(output, outputW, outputB, name="predictions")
#             self.binaryPreds = tf.cast(tf.greater_equal(self.predictions, 0.5), tf.float32, name="binaryPreds")
            self.scores = tf.nn.xw_plus_b(output, outputW, outputB, name='scores')
            self.predictions = tf.argmax(self.scores, 1, name='predictions')
    
        # 计算二元交叉熵损失
        with tf.name_scope("loss"):
            # update
            losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.inputY)
            self.loss = tf.reduce_mean(losses) + config.model.l2RegLambda * l2Loss
            
        # accuracy
        with tf.name_scope('accuracy'):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.inputY, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32), name='accuracy')
    
    def attention(self, H):
        """
        利用Attention机制得到句子的向量表示
        """
        # 获得最后一层LSTM的神经元数量
        hiddenSize = config.model.hiddenSizes[-1]
        
        # 初始化一个权重向量，是可训练的参数
        W = tf.Variable(tf.random_normal([hiddenSize], stddev=0.1))
        
        # 对Bi-LSTM的输出用激活函数做非线性转换
        M = tf.tanh(H)
        
        # 对W和M做矩阵运算，W=[batch_size, time_step, hidden_size]，计算前做维度转换成[batch_size * time_step, hidden_size]
        # newM = [batch_size, time_step, 1]，每一个时间步的输出由向量转换成一个数字
        newM = tf.matmul(tf.reshape(M, [-1, hiddenSize]), tf.reshape(W, [-1, 1]))
        
        # 对newM做维度转换成[batch_size, time_step]
        restoreM = tf.reshape(newM, [-1, config.sequenceLength])
        
        # 用softmax做归一化处理[batch_size, time_step]
        self.alpha = tf.nn.softmax(restoreM)
        
        # 利用求得的alpha的值对H进行加权求和，用矩阵运算直接操作
        r = tf.matmul(tf.transpose(H, [0, 2, 1]), tf.reshape(self.alpha, [-1, config.sequenceLength, 1]))
        
        # 将三维压缩成二维sequeezeR=[batch_size, hidden_size]
        sequeezeR = tf.squeeze(r)
        
        sentenceRepren = tf.tanh(sequeezeR)
        
        # 对Attention的输出可以做dropout处理
        output = tf.nn.dropout(sentenceRepren, self.dropoutKeepProb)
        
        return output

In [20]:
# 定义性能指标函数

def mean(item):
    return sum(item) / len(item)


def genMetrics(trueY, predY, binaryPredY):
    """
    生成acc和auc值
    """
    auc = 0
    try:
        auc = roc_auc_score(trueY, predY)
    except ValueError:
        pass
    accuracy = accuracy_score(trueY, binaryPredY)
    precision = precision_score(trueY, binaryPredY)
    recall = recall_score(trueY, binaryPredY)
    
    return round(accuracy, 4), round(auc, 4), round(precision, 4), round(recall, 4)

In [20]:
# 训练模型
import os
import shutil
# 生成训练集和验证集
trainReviews = data.trainReviews
trainLabels = data.trainLabels
evalReviews = data.evalReviews
evalLabels = data.evalLabels

wordEmbedding = data.wordEmbedding

# 定义计算图
with tf.Graph().as_default():

    session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
    session_conf.gpu_options.allow_growth=True
    session_conf.gpu_options.per_process_gpu_memory_fraction = 0.9  # 配置gpu占用率  

    sess = tf.Session(config=session_conf)
    
    # 定义会话
    with sess.as_default():
        lstm = BiLSTMAttention(config, wordEmbedding)
        
        globalStep = tf.Variable(0, name="globalStep", trainable=False)
        # 定义优化函数，传入学习速率参数
        optimizer = tf.train.AdamOptimizer(config.training.learningRate)
        # 计算梯度,得到梯度和变量
        gradsAndVars = optimizer.compute_gradients(lstm.loss)
        # 将梯度应用到变量下，生成训练器
        trainOp = optimizer.apply_gradients(gradsAndVars, global_step=globalStep)
        
        # 用summary绘制tensorBoard
        gradSummaries = []
        for g, v in gradsAndVars:
            if g is not None:
                tf.summary.histogram("{}/grad/hist".format(v.name), g)
                tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
        
        outDir = os.path.abspath(os.path.join(os.path.curdir, "summarys"))
        print("Writing to {}\n".format(outDir))
        
        lossSummary = tf.summary.scalar("loss", lstm.loss)
        acc_summary = tf.summary.scalar('accuracy', lstm.accuracy)
        summaryOp = tf.summary.merge_all()
        
        trainSummaryDir = os.path.join(outDir, "train")
        trainSummaryWriter = tf.summary.FileWriter(trainSummaryDir, sess.graph)
        
        evalSummaryDir = os.path.join(outDir, "eval")
        evalSummaryWriter = tf.summary.FileWriter(evalSummaryDir, sess.graph)
        
        
        # 初始化所有变量
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=5)
        
        # 保存模型的一种方式，保存为pb文件
        shutil.rmtree('../model/Bi-LSTM-Attention/')
        builder = tf.saved_model.builder.SavedModelBuilder("../model/Bi-LSTM-Attention/savedModel")
        sess.run(tf.global_variables_initializer())

        def trainStep(batchX, batchY):
            """
            训练函数
            """   
            feed_dict = {
              lstm.inputX: batchX,
              lstm.inputY: batchY,
              lstm.dropoutKeepProb: config.model.dropoutKeepProb
            }
#             _, summary, step, loss, predictions, binaryPreds = sess.run(
#                 [trainOp, summaryOp, globalStep, lstm.loss, lstm.predictions, lstm.binaryPreds],
#                 feed_dict)
            _, summaries,step, loss, accuracy = sess.run(
                [trainOp, summaryOp, globalStep, lstm.loss, lstm.accuracy],
                feed_dict=feed_dict
            )
            timeStr = datetime.datetime.now().isoformat()
#             acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
#             print("{}, step: {}, loss: {}, acc: {}, auc: {}, precision: {}, recall: {}".format(timeStr, step, loss, acc, auc, precision, recall))
            print("{}: step {}, loss {:g}, acc {:g}".format(timeStr, step, loss, accuracy))
            trainSummaryWriter.add_summary(summaries, step)

        def devStep(batchX, batchY, writer=None):
            """
            验证函数
            """
            feed_dict = {
              lstm.inputX: batchX,
              lstm.inputY: batchY,
              lstm.dropoutKeepProb: 1.0
            }
#             summary, step, loss, predictions, binaryPreds = sess.run(
#                 [summaryOp, globalStep, lstm.loss, lstm.predictions, lstm.binaryPreds],
#                 feed_dict)
            
#             acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            
#             evalSummaryWriter.add_summary(summary, step)
            
#             return loss, acc, auc, precision, recall
            _, summaries,step, loss, accuracy = sess.run(
                [trainOp, summaryOp, globalStep, lstm.loss, lstm.accuracy],
                feed_dict=feed_dict
            )
            
            #acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            timeStr = datetime.datetime.now().isoformat()
            print("Evaluation:{}: step {}, loss {:g}, acc {:g}".format(timeStr, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)
        
        for i in range(config.training.epoches):
            # 训练模型
            print("start training model")
            for batchTrain in nextBatch(trainReviews, trainLabels, config.batchSize):
                trainStep(batchTrain[0], batchTrain[1])

                currentStep = tf.train.global_step(sess, globalStep) 
                if currentStep % config.training.evaluateEvery == 0:
                    print("\nEvaluation:")
                    
                    for batchEval in nextBatch(evalReviews, evalLabels, config.batchSize):
                        devStep(evalReviews, evalLabels, writer=evalSummaryWriter)
                        
            
                if currentStep % config.training.checkpointEvery == 0:
                    # 保存模型的另一种方法，保存checkpoint文件
                    path = saver.save(sess, "../model/Bi-LSTM-Attention/model/my-model", global_step=currentStep)
                    print("Saved model checkpoint to {}\n".format(path))
                    
#         inputs = {"inputX": tf.saved_model.utils.build_tensor_info(lstm.inputX),
#                   "keepProb": tf.saved_model.utils.build_tensor_info(lstm.dropoutKeepProb)}

#         outputs = {"binaryPreds": tf.saved_model.utils.build_tensor_info(lstm.binaryPreds)}

#         prediction_signature = tf.saved_model.signature_def_utils.build_signature_def(inputs=inputs, outputs=outputs,
#                                                                                       method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME)
#         legacy_init_op = tf.group(tf.tables_initializer(), name="legacy_init_op")
#         builder.add_meta_graph_and_variables(sess, [tf.saved_model.tag_constants.SERVING],
#                                             signature_def_map={"predict": prediction_signature}, legacy_init_op=legacy_init_op)

#         builder.save()

INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name bi-lstm0/fw/lstm_cell/kernel:0/grad/hist is illegal; using bi-lstm0/fw/lstm_cell/kernel_0/grad/hist instead.
INFO:tensorflow:Summary name bi-lstm0/fw/lstm_cell/kernel:0/grad/sparsity is illegal; using bi-lstm0/fw/lstm_cell/kernel_0/grad/sparsity instead.
INFO:tensorflow:Summary name bi-lstm0/fw/lstm_cell/bias:0/grad/hist is illegal; using bi-lstm0/fw/lstm_cell/bias_0/grad/hist instead.
INFO:tensorflow:Summary name bi-lstm0/fw/lstm_cell/bias:0/grad/sparsity is illegal; using bi-lstm0/fw/lstm_cell/bias_0/grad/sparsity instead.
INFO:tensorflow:Summary name bi-lstm0/bw/lstm_cell/kernel:0/grad/hist is illegal; using bi-lstm0/bw/lstm_cell/kernel_0/grad/hist instead.
INFO:tensorflow:Summary name bi-lstm0/bw/lstm_cell/kernel:0/grad/sparsity is illegal; 

2019-04-18T10:30:41.300593: step 69, loss 3.3822, acc 0
2019-04-18T10:30:49.114683: step 70, loss 3.34916, acc 0

Evaluation:
Evaluation:2019-04-18T10:31:29.406932: step 71, loss 1.04023, acc 0.557
Evaluation:2019-04-18T10:32:07.374401: step 72, loss 1.05551, acc 0.562
Evaluation:2019-04-18T10:32:46.436940: step 73, loss 1.06378, acc 0.551
Evaluation:2019-04-18T10:33:25.554333: step 74, loss 1.06499, acc 0.546
Evaluation:2019-04-18T10:34:03.280447: step 75, loss 1.05979, acc 0.549
Evaluation:2019-04-18T10:34:40.969658: step 76, loss 1.04921, acc 0.548
Evaluation:2019-04-18T10:35:18.110336: step 77, loss 1.03436, acc 0.549
Saved model checkpoint to ../model/Bi-LSTM-Attention/model/my-model-70

2019-04-18T10:35:29.451010: step 78, loss 3.26976, acc 0
2019-04-18T10:35:38.107858: step 79, loss 3.29771, acc 0
2019-04-18T10:35:45.968838: step 80, loss 3.36189, acc 0

Evaluation:
Evaluation:2019-04-18T10:36:30.442417: step 81, loss 1.04213, acc 0.659
Evaluation:2019-04-18T10:37:15.464022: ste

Evaluation:2019-04-18T11:25:16.610832: step 176, loss 0.222357, acc 0.999
Evaluation:2019-04-18T11:25:55.819978: step 177, loss 0.199274, acc 0.999
Saved model checkpoint to ../model/Bi-LSTM-Attention/model/my-model-170

2019-04-18T11:26:07.922614: step 178, loss 3.31153, acc 0
2019-04-18T11:26:15.938178: step 179, loss 3.1325, acc 0
2019-04-18T11:26:24.678804: step 180, loss 3.13228, acc 0.0078125

Evaluation:
Evaluation:2019-04-18T11:27:04.170197: step 181, loss 0.221092, acc 1
Evaluation:2019-04-18T11:27:43.374357: step 182, loss 0.264569, acc 0.999
Evaluation:2019-04-18T11:28:25.465797: step 183, loss 0.301743, acc 0.998
Evaluation:2019-04-18T11:29:04.261051: step 184, loss 0.316897, acc 0.997
Evaluation:2019-04-18T11:29:43.406367: step 185, loss 0.304886, acc 0.995
Evaluation:2019-04-18T11:30:22.272158: step 186, loss 0.2736, acc 0.995
Evaluation:2019-04-18T11:31:01.315746: step 187, loss 0.234461, acc 0.997
Saved model checkpoint to ../model/Bi-LSTM-Attention/model/my-model-180



2019-04-18T12:23:44.799852: step 279, loss 2.23875, acc 0.109375
2019-04-18T12:23:52.609967: step 280, loss 2.0972, acc 0.179688

Evaluation:
Evaluation:2019-04-18T12:24:33.441774: step 281, loss 0.050577, acc 0.998
Evaluation:2019-04-18T12:25:13.433827: step 282, loss 0.0739406, acc 0.995
Evaluation:2019-04-18T12:25:53.630333: step 283, loss 0.0876724, acc 0.988
Evaluation:2019-04-18T12:26:34.439203: step 284, loss 0.0790844, acc 0.991
Evaluation:2019-04-18T12:27:14.615764: step 285, loss 0.0578904, acc 0.997
Evaluation:2019-04-18T12:27:54.077235: step 286, loss 0.0393936, acc 0.998
Evaluation:2019-04-18T12:28:34.251803: step 287, loss 0.0278246, acc 0.998
Saved model checkpoint to ../model/Bi-LSTM-Attention/model/my-model-280

2019-04-18T12:28:43.558914: step 288, loss 2.76914, acc 0.0390625
2019-04-18T12:28:52.153928: step 289, loss 2.34858, acc 0.101562
start training model
2019-04-18T12:29:00.184452: step 290, loss 2.20186, acc 0.15625

Evaluation:
Evaluation:2019-04-18T12:29:39.9

Evaluation:2019-04-18T13:17:20.914611: step 384, loss 0.0156322, acc 1
Evaluation:2019-04-18T13:18:01.278672: step 385, loss 0.0158676, acc 1
Evaluation:2019-04-18T13:18:42.370781: step 386, loss 0.0156801, acc 1
Evaluation:2019-04-18T13:19:24.158035: step 387, loss 0.0151514, acc 1
Saved model checkpoint to ../model/Bi-LSTM-Attention/model/my-model-380

2019-04-18T13:19:33.992736: step 388, loss 2.29336, acc 0.179688
2019-04-18T13:19:42.191810: step 389, loss 2.06356, acc 0.195312
2019-04-18T13:19:50.623265: step 390, loss 2.00098, acc 0.195312

Evaluation:
Evaluation:2019-04-18T13:20:32.058456: step 391, loss 0.0186418, acc 1
Evaluation:2019-04-18T13:21:12.841396: step 392, loss 0.021822, acc 1
Evaluation:2019-04-18T13:21:54.085102: step 393, loss 0.0244704, acc 1
Evaluation:2019-04-18T13:22:35.104409: step 394, loss 0.0255945, acc 1
Evaluation:2019-04-18T13:23:15.672920: step 395, loss 0.0249659, acc 1
Evaluation:2019-04-18T13:23:56.032989: step 396, loss 0.0231145, acc 1
Evaluation